In [82]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import os
import time

get_url_final function

In [81]:
#this function just gets the html code that we need to scrape for data, updated version that is not specific to salah 
def get_url_final(code, year_range, category, player):
    base_url = 'https://fbref.com/en/players/{}/matchlogs/{}/c9/{}/{}-Match-Logs'
    url = base_url.format(code, year_range, category, player)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    html_filtered = soup.find('tbody')
    return(html_filtered)

get_dat, get_date and get_matchweek functions

In [80]:
def get_dat(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    return (coldat)   

def get_date(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('th', {'data-stat': col}).text)
    return (coldat)   

def get_matchweek(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    coldat = [coldat.replace('Matchweek ', '') for coldat in coldat]
    return(coldat)

get_data function

In [75]:
#now, we want to modify the function above, to allow us to get any players data that we want, not just salah

def get_data(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    
    #this if statement is checking whether or not this player actually has data for the particular year. In the case that they do not actually have
    #data for this year, just return an empty dataframe 
    if not data_summary:
        empty = pd.DataFrame()
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

In [77]:
#this is just to use the get_data function to create a representative dataframe, with the purpose being just to get the column names. This is why 
#we have to specify another get_data function below (get_data_final), which implements functionality that deals with players who have empty pages in 
#Fbref
test = get_data('66b76d44', '2022-2023', 'Emi-Buendia', 'Emiliano_Buen')
column_names = test.columns

In [97]:
#this is the final version of get_data, which includes additional functionality to deal with players who hav eempty pages in Fbref
def get_data_final(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    
    #this if statement is checking whether or not this player actually has data for the particular year. In the case that they do not actually have
    #data for this year, just return an empty dataframe 
    if not data_summary:
        empty = pd.DataFrame(columns = column_names)
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

get_premgames function

In [96]:
def get_premgames (code, player):
    base_url = f'https://fbref.com/en/players/{code}/{player}'
    html = requests.get(base_url).text
    soup = BeautifulSoup(html, 'lxml')
    summary = soup.find('table', class_='stats_table sortable min_width')
    table = summary.find('tbody')
    
    comp = table.find_all('td', attrs={'data-stat': 'comp_level'})
    comp_text = [cell.get_text() for cell in comp]
    games = table.find_all('td', attrs={'data-stat': 'games'})
    games_text = [cell.get_text() for cell in games]
    season = table.find_all('th', attrs={'data-stat': 'year_id'})
    season_text  = [cell.get_text() for cell in season]
    
    df = pd.DataFrame({
        'Season': season_text,
        'Competition': comp_text, 
        'Games Played': games_text
    })

    season_list = ('2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018')
    total_games = df[(df['Competition'] == '1. Premier League') & (df['Season'].isin(season_list))]['Games Played']
    total_games = pd.to_numeric(total_games)
    prem_games = total_games.sum()
    return(prem_games)

This code specifies the location to place the .csv file into

In [25]:
nest_folder = ("Player_Data")
os.makedirs(nest_folder, exist_ok = True)

## Now, we actually use the functions above to get the necessary data 

### Diogo Jota

In [35]:
jota_2324_finaldf = get_data('178ae8f8', '2023-2024', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [9]:
jota_2223_finaldf = get_data('178ae8f8', '2022-2023', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [10]:
jota_2122_finaldf = get_data('178ae8f8', '2021-2022', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [12]:
jota_2021_finaldf = get_data('178ae8f8', '2020-2021', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [6]:
jota_1920_finaldf = get_data('178ae8f8', '2019-2020', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [7]:
jota_1819_finaldf = get_data('178ae8f8', '2018-2019', 'Diogo-Jota',  "Diogo_Teixeira da Silva")

In [26]:
jota_finaldat = pd.concat([jota_1819_finaldf, jota_1920_finaldf, jota_2021_finaldf, jota_2122_finaldf
                            , jota_2223_finaldf, jota_2324_finaldf], join = 'inner', ignore_index = True)
jota_finaldat.to_csv(os.path.join(nest_folder, "jota_finaldat.csv"))

### Luis Diaz

In [27]:
diaz_2324_finaldf = get_data('4a1a9578', '2023-2024', 'Luis-Diaz',  "Luis_Díaz")

In [28]:
diaz_2223_finaldf = get_data('4a1a9578', '2022-2023', 'Luis-Diaz',  "Luis_Díaz")

In [29]:
diaz_2122_finaldf = get_data('4a1a9578', '2021-2022', 'Luis-Diaz',  "Luis_Díaz")

In [30]:
diaz_finaldat = pd.concat([diaz_2122_finaldf, diaz_2223_finaldf, diaz_2324_finaldf], join = 'inner', ignore_index = True)
diaz_finaldat.to_csv(os.path.join(nest_folder, "diaz_finaldat.csv"))

### Cody Gakpo

In [31]:
gakpo_2324_finaldf = get_data('1971591f', '2023-2024', 'Cody-Gakpo',  "Cody_Gakpo")

In [32]:
gakpo_2223_finaldf = get_data('1971591f', '2022-2023', 'Cody-Gakpo',  "Cody_Gakpo")

In [33]:
gakpo_finaldat = pd.concat([gakpo_2223_finaldf, gakpo_2324_finaldf], join = 'inner', ignore_index = True)
gakpo_finaldat.to_csv(os.path.join(nest_folder, "gakpo_finaldat.csv"))

### Darwin Nunez

In [36]:
darwin_2324_finaldf = get_data('4d77b365', '2023-2024', 'Darwin-Nunez',  "Darwin_Núñez Ribeiro")

In [38]:
darwin_2223_finaldf = get_data('4d77b365', '2022-2023', 'Darwin-Nunez',  "Darwin")

In [39]:
darwin_finaldat = pd.concat([darwin_2223_finaldf, darwin_2324_finaldf], join = 'inner', ignore_index = True)
darwin_finaldat.to_csv(os.path.join(nest_folder, "darwin_finaldat.csv"))

# Updated Function
This new function compile_dat allows us to get all of the data for a single player in one function, instead of having to do it one by one as can be seen above. 

In [136]:
def compile_dat (code, player, fpl_name, checkgames):
    #these are the seasons for which we have fpl data 
    year_list = ('2023-24', '2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18')
    
    #active_years is a list that will store the seasons that a given player was active. The for loop will check to see which years a player was active
    #in the prem, and will append this year to the list
    active_years = []
    for year in year_list: 
        directory = f'Fantasy-Premier-League/data/{year}/players'
        files = os.listdir(directory)
        csv_file = [f for f in files if fpl_name in f]
        if csv_file:
            update_year = year[:5] + '20' + year[5:]
            active_years.append(update_year)
    
    #create an empty dictionary to store all of the dataframes 
    dataframes = {}
    
    #for each season a player was active, the get_data function from above is uesd to add the data for that season to the dataframes dictionary
    for year in active_years: 
        dataframes[year] = get_data_final(code, year, player, fpl_name)
        #1 minute pause in between each iteration, to ensure that we don't get banned from FBref
        time.sleep(60)
    
    #concatenate all of the dataframes, and return one final concatenated dataframe
    finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)
    
    if checkgames:
        games_played = get_premgames(code, player)
        if games_played == finaldf.shape[0]:
            return(finaldf)
        else:
            return(print('The number of Premier League games played by ' + player + '(' + str(games_played) +  ') does not match the number of rows in the dataframe ('+ str(finaldf.shape[0]) + ')'))
    else: 
        return(finaldf)

#note that the to_csv code is not included in this function, because there are certain players whose names were changed in between seasons 
#(e.g. Diogo Teixeira da Silva to Diogo Jota). In these cases, we need to use the compile function twice to get all of the data, before concatenating
#and then finally exporting to csv 

## Arsenal

In [43]:
saka_finaldat = compile_dat('bc7dc64d', 'Bukayo-Saka', 'Bukayo')
saka_finaldat.to_csv(os.path.join(nest_folder, "saka_finaldat.csv"))

In [45]:
havertz_finaldat = compile_dat('fed7cb61', 'Kai-Havertz', 'Kai_Havertz')
havertz_finaldat.to_csv(os.path.join(nest_folder, "havertz_finaldat.csv"))

In [50]:
jesus_finaldat = compile_dat('b66315ae', 'Gabriel-Jesus', 'de Jesus')
jesus_finaldat.to_csv(os.path.join(nest_folder, "jesus_finaldat.csv"))

In [47]:
trossard_finaldat = compile_dat('38ceb24a', 'Leandro-Trossard', 'Leandro_Trossard')
trossard_finaldat.to_csv(os.path.join(nest_folder, "trossard_finaldat.csv"))

In [51]:
martinelli_finaldat = compile_dat('48a5a5d6', 'Gabriel-Martinelli', 'Martinelli')
martinelli_finaldat.to_csv(os.path.join(nest_folder, "martinelli_finaldat.csv"))

In [52]:
odegaard_finaldat = compile_dat('79300479', 'Martin-Odegaard', 'Ødegaard')
odegaard_finaldat.to_csv(os.path.join(nest_folder, "odegaard_finaldat.csv"))

In [53]:
sterling_finaldat = compile_dat('b400bde0', 'Raheem-Sterling', 'Raheem_Sterling')
sterling_finaldat.to_csv(os.path.join(nest_folder, "sterling_finaldat.csv"))

In [88]:
jorginho_finaldat = compile_dat('45db685d', 'Jorginho', 'Jorge Luiz_Frello', checkgames = True)
if isinstance(jorginho_finaldat, pd.DataFrame):
    jorginho_finaldat.to_csv(os.path.join(nest_folder, "jorginho_finaldat.csv"))

In [100]:
rice_finaldat = compile_dat('1c7012b8', 'Declan-Rice', 'Declan_Rice', checkgames = True)
if isinstance(rice_finaldat, pd.DataFrame):
    rice_finaldat.to_csv(os.path.join(nest_folder, "rice_finaldat.csv"))

In [90]:
partey_finaldat = compile_dat('529f49ab', 'Thomas-Partey', 'Thomas_Partey', checkgames = True)
if isinstance(partey_finaldat, pd.DataFrame):
    partey_finaldat.to_csv(os.path.join(nest_folder, "partey_finaldat.csv"))

## Aston Villa

In [61]:
duran_finaldat = compile_dat('414184f7', 'Jhon-Duran', 'Jhon', checkgames=True)
duran_finaldat.to_csv(os.path.join(nest_folder, "duran_finaldat.csv"))

In [63]:
watkins_finaldat = compile_dat('aed3a70f', 'Ollie-Watkins', 'Ollie_Watkins', checkgames=True)
if isinstance(watkins_finaldat, pd.DataFrame):
    watkins_finaldat.to_csv(os.path.join(nest_folder, "watkins_finaldat.csv"))

In [64]:
ramsey_finaldat = compile_dat('1544f145', 'Jacob-Ramsey', 'Jacob_Ramsey', checkgames = True)
if isinstance(ramsey_finaldat, pd.DataFrame):
    ramsey_finaldat.to_csv(os.path.join(nest_folder, "ramsey_finaldat.csv"))

In [68]:
mcginn_finaldat = compile_dat('90f91999', 'John-McGinn', 'John_McGinn', checkgames = False)
if isinstance(mcginn_finaldat, pd.DataFrame):
    mcginn_finaldat.to_csv(os.path.join(nest_folder, "mcginn_finaldat.csv"))

In [66]:
bailey_finaldat = compile_dat('3a233281', 'Leon-Bailey', 'Leon_Bailey', checkgames = True)
if isinstance(bailey_finaldat, pd.DataFrame):
    bailey_finaldat.to_csv(os.path.join(nest_folder, "bailey_finaldat.csv"))

In [84]:
buendia_finaldat = compile_dat('66b76d44', 'Emi-Buendia', 'Emiliano_Buen', checkgames = True)
if isinstance(buendia_finaldat, pd.DataFrame):
    buendia_finaldat.to_csv(os.path.join(nest_folder, "buendia_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/2405533476.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [69]:
tielemans_finaldat = compile_dat('56f7a928', 'Youri-Tielemans', 'Youri_Tielemans', checkgames = True)
if isinstance(tielemans_finaldat, pd.DataFrame):
    tielemans_finaldat.to_csv(os.path.join(nest_folder, "tielemans_finaldat.csv"))

In [92]:
barkley_finaldat = compile_dat('3a24769f', 'Ross-Barkley', 'Ross_Barkley', checkgames = False)
if isinstance(barkley_finaldat, pd.DataFrame):
    barkley_finaldat.to_csv(os.path.join(nest_folder, "barkley_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/2561086557.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [86]:
onana_finaldat = compile_dat('828657ff', 'Amadou-Onana', 'Amadou_Onana', checkgames = True)
if isinstance(onana_finaldat, pd.DataFrame):
    onana_finaldat.to_csv(os.path.join(nest_folder, "onana_finaldat.csv"))

In [87]:
rogers_finaldat = compile_dat('2e5915f1', 'Morgan-Rogers', 'Morgan_Rogers', checkgames = True)
if isinstance(rogers_finaldat, pd.DataFrame):
    rogers_finaldat.to_csv(os.path.join(nest_folder, "rogers_finaldat.csv"))

## Bournemouth

In [101]:
unal_finaldat = compile_dat('f8eca1b6', 'Enes-Unal', 'Enes', checkgames = True)
if isinstance(unal_finaldat, pd.DataFrame):
    unal_finaldat.to_csv(os.path.join(nest_folder, "unal_finaldat.csv"))

In [102]:
semenyo_finaldat = compile_dat('efd2ec23', 'Antoine-Semenyo', 'Antoine_Semenyo', checkgames = True)
if isinstance(semenyo_finaldat, pd.DataFrame):
    semenyo_finaldat.to_csv(os.path.join(nest_folder, "semenyo_finaldat.csv"))

In [103]:
sinisterra_finaldat = compile_dat('8e16dd48', 'Luis-Sinisterra', 'Luis_Sinisterra', checkgames = True)
if isinstance(sinisterra_finaldat, pd.DataFrame):
    sinisterra_finaldat.to_csv(os.path.join(nest_folder, "sinisterra_finaldat.csv"))

In [104]:
tavernier_finaldat = compile_dat('5c0da4a4', 'Marcus-Tavernier', 'Marcus_Tavernier', checkgames = True)
if isinstance(tavernier_finaldat, pd.DataFrame):
    tavernier_finaldat.to_csv(os.path.join(nest_folder, "tavernier_finaldat.csv"))

In [105]:
cook_finaldat = compile_dat('2afc7272', 'Lewis-Cook', 'Lewis_Cook', checkgames = True)
if isinstance(cook_finaldat, pd.DataFrame):
    cook_finaldat.to_csv(os.path.join(nest_folder, "cook_finaldat.csv"))

In [106]:
kluivert_finaldat = compile_dat('4c3a6744', 'Justin-Kluivert', 'Justin_Kluivert', checkgames = True)
if isinstance(kluivert_finaldat, pd.DataFrame):
    kluivert_finaldat.to_csv(os.path.join(nest_folder, "kluivert_finaldat.csv"))

In [107]:
dango_finaldat = compile_dat('2f9e4435', 'Dango-Ouattara', 'Dango_Ouattara', checkgames = True)
if isinstance(dango_finaldat, pd.DataFrame):
    dango_finaldat.to_csv(os.path.join(nest_folder, "dango_finaldat.csv"))

In [108]:
christie_finaldat = compile_dat('26ce2263', 'Ryan-Christie', 'Ryan_Christie', checkgames = True)
if isinstance(christie_finaldat, pd.DataFrame):
    christie_finaldat.to_csv(os.path.join(nest_folder, "christie_finaldat.csv"))

In [109]:
scott_finaldat = compile_dat('104d0bb8', 'Alex-Scott', 'Alex_Scott', checkgames = True)
if isinstance(scott_finaldat, pd.DataFrame):
    scott_finaldat.to_csv(os.path.join(nest_folder, "scott_finaldat.csv"))

In [110]:
billing_finaldat = compile_dat('d328a254', 'Philip-Billing', 'Philip_Billing', checkgames = True)
if isinstance(billing_finaldat, pd.DataFrame):
    billing_finaldat.to_csv(os.path.join(nest_folder, "billing_finaldat.csv"))

In [111]:
adams_finaldat = compile_dat('2b09d998', 'Tyler-Adams', 'Tyler_Adams', checkgames = True)
if isinstance(adams_finaldat, pd.DataFrame):
    adams_finaldat.to_csv(os.path.join(nest_folder, "adams_finaldat.csv"))

In [113]:
brooks_finaldat = compile_dat('dc4cae05', 'David-Brooks', 'David_Brooks', checkgames = True)
if isinstance(brooks_finaldat, pd.DataFrame):
    brooks_finaldat.to_csv(os.path.join(nest_folder, "brooks_finaldat.csv"))

## Brentford

In [114]:
toney_finaldat = compile_dat('e09f279b', 'Ivan-Toney', 'Ivan_Toney', checkgames = True)
if isinstance(toney_finaldat, pd.DataFrame):
    toney_finaldat.to_csv(os.path.join(nest_folder, "toney_finaldat.csv"))

In [115]:
wissa_finaldat = compile_dat('2500cef9', 'Yoane-Wissa', 'Yoane_Wissa', checkgames = True)
if isinstance(wissa_finaldat, pd.DataFrame):
    wissa_finaldat.to_csv(os.path.join(nest_folder, "wissa_finaldat.csv"))

In [116]:
mbeumo_finaldat = compile_dat('6afaebf2', 'Bryan-Mbeumo', 'Bryan_Mbeumo', checkgames = True)
if isinstance(mbeumo_finaldat, pd.DataFrame):
    mbeumo_finaldat.to_csv(os.path.join(nest_folder, "mbeumo_finaldat.csv"))

In [117]:
jensen_finaldat = compile_dat('0f134faf', 'Mathias-Jensen', 'Mathias_Jensen', checkgames = True)
if isinstance(jensen_finaldat, pd.DataFrame):
    jensen_finaldat.to_csv(os.path.join(nest_folder, "jensen_finaldat.csv"))

In [118]:
schade_finaldat = compile_dat('52afb588', 'Kevin-Schade', 'Kevin_Schade', checkgames = True)
if isinstance(schade_finaldat, pd.DataFrame):
    schade_finaldat.to_csv(os.path.join(nest_folder, "schade_finaldat.csv"))

In [119]:
janelt_finaldat = compile_dat('8449d35e', 'Vitaly-Janelt', 'Vitaly_Janelt', checkgames = True)
if isinstance(janelt_finaldat, pd.DataFrame):
    janelt_finaldat.to_csv(os.path.join(nest_folder, "janelt_finaldat.csv"))

In [120]:
damsgaard_finaldat = compile_dat('215f3907', 'Mikkel-Damsgaard', 'Mikkel_Damsgaard', checkgames = True)
if isinstance(damsgaard_finaldat, pd.DataFrame):
    damsgaard_finaldat.to_csv(os.path.join(nest_folder, "damsgaard_finaldat.csv"))

In [121]:
norgaard_finaldat = compile_dat('df0a4c90', 'Christian-Norgaard', 'Christian_Nø', checkgames = True)
if isinstance(norgaard_finaldat, pd.DataFrame):
    norgaard_finaldat.to_csv(os.path.join(nest_folder, "norgaard_finaldat.csv"))

In [122]:
lewispotter_finaldat = compile_dat('41f08ac8', 'Keane-Lewis-Potter', 'Keane_Lewis', checkgames = True)
if isinstance(lewispotter_finaldat, pd.DataFrame):
    lewispotter_finaldat.to_csv(os.path.join(nest_folder, "lewispotter_finaldat.csv"))

## Brighton

In [123]:
welbeck_finaldat = compile_dat('ce5143da', 'Danny-Welbeck', 'Danny_Welbeck', checkgames = True)
if isinstance(welbeck_finaldat, pd.DataFrame):
    welbeck_finaldat.to_csv(os.path.join(nest_folder, "welbeck_finaldat.csv"))

In [124]:
jpedro_finaldat = compile_dat('e8832875', 'Joao-Pedro', 'Junqueira', checkgames = True)
if isinstance(jpedro_finaldat, pd.DataFrame):
    jpedro_finaldat.to_csv(os.path.join(nest_folder, "jpedro_finaldat.csv"))

In [125]:
ferguson_finaldat = compile_dat('4596da74', 'Evan-Ferguson', 'Evan_Ferguson', checkgames = True)
if isinstance(ferguson_finaldat, pd.DataFrame):
    ferguson_finaldat.to_csv(os.path.join(nest_folder, "ferguson_finaldat.csv"))

In [126]:
mitoma_finaldat = compile_dat('74618572', 'Kaoru-Mitoma', 'Kaoru_Mitoma', checkgames = True)
if isinstance(mitoma_finaldat, pd.DataFrame):
    mitoma_finaldat.to_csv(os.path.join(nest_folder, "mitoma_finaldat.csv"))

In [127]:
adingra_finaldat = compile_dat('4dcec659', 'Simon-Adingra', 'Simon_Adingra', checkgames = True)
if isinstance(adingra_finaldat, pd.DataFrame):
    adingra_finaldat.to_csv(os.path.join(nest_folder, "adingra_finaldat.csv"))

In [128]:
hinshelwood_finaldat = compile_dat('e98211e7', 'Jack-Hinshelwood', 'Jack_Hinshelwood', checkgames = True)
if isinstance(hinshelwood_finaldat, pd.DataFrame):
    hinshelwood_finaldat.to_csv(os.path.join(nest_folder, "hinshelwood_finaldat.csv"))

In [129]:
milner_finaldat = compile_dat('2f90f6b8', 'James-Milner', 'James_Milner', checkgames = True)
if isinstance(milner_finaldat, pd.DataFrame):
    milner_finaldat.to_csv(os.path.join(nest_folder, "milner_finaldat.csv"))

In [130]:
enciso_finaldat = compile_dat('9cfbad36', 'Julio-Enciso', 'Julio_Enciso', checkgames = True)
if isinstance(enciso_finaldat, pd.DataFrame):
    enciso_finaldat.to_csv(os.path.join(nest_folder, "enciso_finaldat.csv"))

In [140]:
march_latedat = compile_dat('bb5fbd2b', 'Solly-March', 'Solly_March', checkgames = False)
march_earlydat = compile_dat('bb5fbd2b', 'Solly-March', 'Solomon_March', checkgames = False)
march_finaldat = pd.concat([march_latedat, march_earlydat], join = "inner", ignore_index = True)
if isinstance(march_finaldat, pd.DataFrame):
    march_finaldat.to_csv(os.path.join(nest_folder, "march_finaldat.csv"))

## Chelsea

In [142]:
jackson_finaldat = compile_dat('9c36ed83', 'Nicolas-Jackson', 'Nicolas_Jackson', checkgames = True)
if isinstance(jackson_finaldat, pd.DataFrame):
    jackson_finaldat.to_csv(os.path.join(nest_folder, "jackson_finaldat.csv"))

In [143]:
palmer_finaldat = compile_dat('dc7f8a28', 'Cole-Palmer', 'Cole_Palmer', checkgames = True)
if isinstance(palmer_finaldat, pd.DataFrame):
    palmer_finaldat.to_csv(os.path.join(nest_folder, "palmer_finaldat.csv"))

In [138]:
madueke_finaldat = compile_dat('bf34eebd', 'Noni-Madueke', 'Noni_Madueke', checkgames = True)
if isinstance(madueke_finaldat, pd.DataFrame):
    madueke_finaldat.to_csv(os.path.join(nest_folder, "madueke_finaldat.csv"))

In [139]:
felix_finaldat = compile_dat('8aafd64f', 'Joao-Felix', 'Sequeira', checkgames = True)
if isinstance(felix_finaldat, pd.DataFrame):
    felix_finaldat.to_csv(os.path.join(nest_folder, "felix_finaldat.csv"))

In [141]:
enzo_finaldat = compile_dat('5ff4ab71', 'Enzo-Fernandez', 'Enzo_Fern', checkgames = True)
if isinstance(enzo_finaldat, pd.DataFrame):
    enzo_finaldat.to_csv(os.path.join(nest_folder, "enzo_finaldat.csv"))

In [156]:
caicedo_finaldat = compile_dat('16264a81', 'Moises-Caicedo', 'Caicedo', checkgames = True)
if isinstance(caicedo_finaldat, pd.DataFrame):
    caicedo_finaldat.to_csv(os.path.join(nest_folder, "caicedo_finaldat.csv"))

In [145]:
neto_finaldat = compile_dat('7ba2eaa9', 'Pedro-Neto', 'Pedro_Lomba', checkgames = True)
if isinstance(neto_finaldat, pd.DataFrame):
    neto_finaldat.to_csv(os.path.join(nest_folder, "neto_finaldat.csv"))

In [146]:
dewsburyhall_finaldat = compile_dat('5c74c0f5', 'Kiernan-Dewsbury-Hall', 'Kiernan_Dewsbury', checkgames = True)
if isinstance(dewsburyhall_finaldat, pd.DataFrame):
    dewsburyhall_finaldat.to_csv(os.path.join(nest_folder, "dewsburyhall_finaldat.csv"))

In [162]:
mudryk_finaldat = compile_dat('049a888d', 'Mykhailo-Mudryk', 'Mykhailo_Mudryk', checkgames = False)
if isinstance(mudryk_finaldat, pd.DataFrame):
    mudryk_finaldat.to_csv(os.path.join(nest_folder, "mudryk_finaldat.csv"))

## Crystal Palace

In [ ]:
nketiah_latedat = compile_dat('a53649b7', 'Eddie-Nketiah', 'Eddie_Nketiah', checkgames = False)
nketiah_earlydat = compile_dat('a53649b7', 'Eddie-Nketiah', 'Edward_Nketiah', checkgames = False)
nketiah_finaldat = pd.concat([nketiah_latedat, nketiah_earlydat], join = "inner", ignore_index = True)
if isinstance(nketiah_finaldat, pd.DataFrame):
    nketiah_finaldat.to_csv(os.path.join(nest_folder, "nketiah_finaldat.csv"))

In [149]:
mateta_finaldat = compile_dat('50e6dc35', 'Jean-Philippe-Mateta', 'Jean-Philippe_Mateta', checkgames = True)
if isinstance(mateta_finaldat, pd.DataFrame):
    mateta_finaldat.to_csv(os.path.join(nest_folder, "mateta_finaldat.csv"))

In [150]:
eze_finaldat = compile_dat('ae4fc6a4', 'Eberechi-Eze', 'Eberechi_Eze', checkgames = True)
if isinstance(eze_finaldat, pd.DataFrame):
    eze_finaldat.to_csv(os.path.join(nest_folder, "eze_finaldat.csv"))

In [151]:
wharton_finaldat = compile_dat('4b542852', 'Adam-Wharton', 'Adam_Wharton', checkgames = True)
if isinstance(wharton_finaldat, pd.DataFrame):
    wharton_finaldat.to_csv(os.path.join(nest_folder, "wharton_finaldat.csv"))

In [152]:
cheick_finaldat = compile_dat('ce4f40c7', 'Cheick-Doucoure', 'Cheick_Doucou', checkgames = True)
if isinstance(cheick_finaldat, pd.DataFrame):
    cheick_finaldat.to_csv(os.path.join(nest_folder, "cheick_finaldat.csv"))

In [153]:
lerma_finaldat = compile_dat('9b5ce51a', 'Jefferson-Lerma', 'Jefferson_Lerma', checkgames = True)
if isinstance(lerma_finaldat, pd.DataFrame):
    lerma_finaldat.to_csv(os.path.join(nest_folder, "lerma_finaldat.csv"))

In [ ]:
ismailasarr_finaldat = compile_dat('bfdb33aa', 'Ismaila-Sarr', 'Ismaila_Sarr', checkgames = True)
if isinstance(ismailasarr_finaldat, pd.DataFrame):
    ismailasarr_finaldat.to_csv(os.path.join(nest_folder, "ismailasarr_finaldat.csv"))

The number of Premier League games played by Ismaila-Sarr(50) does not match the number of rows in the dataframe (22)


In [155]:
hughes_finaldat = compile_dat('a0666d3e', 'Will-Hughes', 'Will_Hughes', checkgames = True)
if isinstance(hughes_finaldat, pd.DataFrame):
    hughes_finaldat.to_csv(os.path.join(nest_folder, "hughes_finaldat.csv"))

In [157]:
schlupp_finaldat = compile_dat('3312f911', 'Jeffrey-Schlupp', 'Jeffrey_Schlupp', checkgames = True)
if isinstance(schlupp_finaldat, pd.DataFrame):
    schlupp_finaldat.to_csv(os.path.join(nest_folder, "schlupp_finaldat.csv"))

## Everton

In [158]:
calvertlewin_finaldat = compile_dat('59e6e5bf', 'Dominic-Calvert-Lewin', 'Dominic_Calvert', checkgames = True)
if isinstance(calvertlewin_finaldat, pd.DataFrame):
    calvertlewin_finaldat.to_csv(os.path.join(nest_folder, "calvertlewin_finaldat.csv"))

In [159]:
beto_finaldat = compile_dat('ed5c0319', 'Beto', 'Norberto Bercique', checkgames = True)
if isinstance(beto_finaldat, pd.DataFrame):
    beto_finaldat.to_csv(os.path.join(nest_folder, "beto_finaldat.csv"))

In [160]:
broja_finaldat = compile_dat('97220da2', 'Armando-Broja', 'Armando_Broja', checkgames = True)
if isinstance(broja_finaldat, pd.DataFrame):
    broja_finaldat.to_csv(os.path.join(nest_folder, "broja_finaldat.csv"))

In [ ]:
mcneil_finaldat = compile_dat('fc15fb84', 'Dwight-McNeil', 'Dwight_McNeil', checkgames = True)
if isinstance(mcneil_finaldat, pd.DataFrame):
    mcneil_finaldat.to_csv(os.path.join(nest_folder, "mcneil_finaldat.csv"))

The number of Premier League games played by Dwight-McNeil(204) does not match the number of rows in the dataframe (205)


In [163]:
gueye_finaldat = compile_dat('72c812f3', 'Idrissa-Gana-Gueye', 'Idrissa_Gueye', checkgames = True)
if isinstance(gueye_finaldat, pd.DataFrame):
    gueye_finaldat.to_csv(os.path.join(nest_folder, "gueye_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [164]:
harrison_finaldat = compile_dat('aa849a12', 'Jack-Harrison', 'Jack_Harrison', checkgames = True)
if isinstance(harrison_finaldat, pd.DataFrame):
    harrison_finaldat.to_csv(os.path.join(nest_folder, "harrison_finaldat.csv"))

In [165]:
doucoure_finaldat = compile_dat('02b29014', 'Abdoulaye-Doucoure', 'Abdoulaye_Douco', checkgames = True)
if isinstance(doucoure_finaldat, pd.DataFrame):
    doucoure_finaldat.to_csv(os.path.join(nest_folder, "doucoure_finaldat.csv"))

In [166]:
garner_finaldat = compile_dat('4e015693', 'James-Garner', 'James_Garner', checkgames = True)
if isinstance(garner_finaldat, pd.DataFrame):
    garner_finaldat.to_csv(os.path.join(nest_folder, "garner_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


## Fulham

In [167]:
muniz_finaldat = compile_dat('a755db8c', 'Rodrigo-Muniz', 'Rodrigo_Muniz', checkgames = True)
if isinstance(muniz_finaldat, pd.DataFrame):
    muniz_finaldat.to_csv(os.path.join(nest_folder, "muniz_finaldat.csv"))

In [170]:
raul_earlydat = compile_dat('b561db50', 'Raul-Jimenez', 'Raúl_Jiménez', checkgames = False)
raul_latedat = compile_dat('b561db50', 'Raul-Jimenez', 'Jiménez', checkgames = False)
raul_finaldat = pd.concat([raul_latedat, raul_earlydat], join = "inner", ignore_index = True)
if isinstance(raul_finaldat, pd.DataFrame):
    raul_finaldat.to_csv(os.path.join(nest_folder, "raul_finaldat.csv"))

In [169]:
carlosvini_finaldat = compile_dat('8b529245', 'Carlos-Vinicius', 'Carlos Vin', checkgames = True)
if isinstance(carlosvini_finaldat, pd.DataFrame):
    carlosvini_finaldat.to_csv(os.path.join(nest_folder, "carlosvini_finaldat.csv"))

In [171]:
adama_finaldat = compile_dat('9a28eba4', 'Adama-Traore', 'Adama', checkgames = True)
if isinstance(adama_finaldat, pd.DataFrame):
    adama_finaldat.to_csv(os.path.join(nest_folder, "adama_finaldat.csv"))

The number of Premier League games played by Adama-Traore(174) does not match the number of rows in the dataframe (163)


In [172]:
smithrowe_finaldat = compile_dat('17695062', 'Emile-Smith-Rowe', 'Emile_Smith', checkgames = True)
if isinstance(smithrowe_finaldat, pd.DataFrame):
    smithrowe_finaldat.to_csv(os.path.join(nest_folder, "smithrowe_finaldat.csv"))

In [173]:
iwobi_finaldat = compile_dat('6ca5ec4b', 'Alex-Iwobi', 'Alex_Iwobi', checkgames = True)
if isinstance(iwobi_finaldat, pd.DataFrame):
    iwobi_finaldat.to_csv(os.path.join(nest_folder, "iwobi_finaldat.csv"))

In [174]:
andreas_earlydat = compile_dat('6639e500', 'Andreas-Pereira', 'Andreas_Pereira', checkgames = False)
andreas_latedat = compile_dat('6639e500', 'Andreas-Pereira', 'Hoelgebaum', checkgames = False)
andreas_finaldat = pd.concat([andreas_latedat, andreas_earlydat], join = "inner", ignore_index = True)
if isinstance(andreas_finaldat, pd.DataFrame):
    andreas_finaldat.to_csv(os.path.join(nest_folder, "andreas_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [175]:
lukic_finaldat = compile_dat('c6e8cf1f', 'Sasa-Lukic', 'Lukic', checkgames = True)
if isinstance(lukic_finaldat, pd.DataFrame):
    lukic_finaldat.to_csv(os.path.join(nest_folder, "lukic_finaldat.csv"))

In [176]:
reed_finaldat = compile_dat('803ae100', 'Harrison-Reed', 'Harrison_Reed', checkgames = True)
if isinstance(reed_finaldat, pd.DataFrame):
    reed_finaldat.to_csv(os.path.join(nest_folder, "reed_finaldat.csv"))

In [177]:
wilson_finaldat = compile_dat('c6dc9ecd', 'Harry-Wilson', 'Harry_Wilson', checkgames = True)
if isinstance(wilson_finaldat, pd.DataFrame):
    wilson_finaldat.to_csv(os.path.join(nest_folder, "wilson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [178]:
nelson_finaldat = compile_dat('c5bdb6e3', 'Reiss-Nelson', 'Reiss_Nelson', checkgames = True)
if isinstance(nelson_finaldat, pd.DataFrame):
    nelson_finaldat.to_csv(os.path.join(nest_folder, "nelson_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


In [179]:
cairney_finaldat = compile_dat('a8748947', 'Tom-Cairney', 'Tom_Cairney', checkgames = True)
if isinstance(cairney_finaldat, pd.DataFrame):
    cairney_finaldat.to_csv(os.path.join(nest_folder, "cairney_finaldat.csv"))

/var/folders/dv/bjx4hkg54x17symh1g3_3jf80000gq/T/ipykernel_60248/3417702198.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  finaldf = pd.concat(dataframes.values(), join = "inner", ignore_index = True)


The number of Premier League games played by Tom-Cairney(108) does not match the number of rows in the dataframe (67)


In [180]:
berge_finaldat = compile_dat('d0b6129f', 'Sander-Berge', 'Sander_Berge', checkgames = True)
if isinstance(berge_finaldat, pd.DataFrame):
    berge_finaldat.to_csv(os.path.join(nest_folder, "berge_finaldat.csv"))

AttributeError: 'NoneType' object has no attribute 'find'

## Leicester

In [ ]:
vardy_finaldat = compile_dat('45963054', 'Jamie-Vardy', 'Jamie_Vardy', checkgames = True)
if isinstance(vardy_finaldat, pd.DataFrame):
    vardy_finaldat.to_csv(os.path.join(nest_folder, "vardy_finaldat.csv"))

In [ ]:
edouard_finaldat = compile_dat('0562b7f1', 'Odsonne-Edouard', 'Odsonne_', checkgames = True)
if isinstance(edouard_finaldat, pd.DataFrame):
    edouard_finaldat.to_csv(os.path.join(nest_folder, "edouard_finaldat.csv"))

In [ ]:
daka_finaldat = compile_dat('ca45605e', 'Patson-Daka', 'Patson_Daka', checkgames = True)
if isinstance(daka_finaldat, pd.DataFrame):
    daka_finaldat.to_csv(os.path.join(nest_folder, "daka_finaldat.csv"))

In [ ]:
buonanotte_finaldat = compile_dat('468a7a91', 'Facundo-Buonanotte', 'Facundo_B', checkgames = True)
if isinstance(buonanotte_finaldat, pd.DataFrame):
    buonanotte_finaldat.to_csv(os.path.join(nest_folder, "buonanotte_finaldat.csv"))

In [ ]:
ndidi_finaldat = compile_dat('6b47c5db', 'Wilfred-Ndidi', checkgames = True)
if isinstance(ndidi_finaldat, pd.DataFrame):
    ndidi_finaldat.to_csv(os.path.join(nest_folder, "ndidi_finaldat.csv"))

In [ ]:
decordovareid_earlydat = compile_dat('0f7533cd', 'Bobby-Reid', 'Bobby_Reid', checkgames = False)
decordovareid_latedat = compile_dat('0f7533cd', 'Bobby-Reid', 'Bobby_De', checkgames = False)
decordovareid_finaldat = pd.concat([decordovareid_latedat, decordovareid_earlydat], join = "inner", ignore_index = True)
if isinstance(decordovareid_finaldat, pd.DataFrame):
    decordovareid_finaldat.to_csv(os.path.join(nest_folder, "decordovareid_finaldat.csv"))

In [ ]:
winks_finaldat = compile_dat('2f7acede', 'Harry-Winks', 'Harry_Winks', checkgames = True)
if isinstance(winks_finaldat, pd.DataFrame):
    winks_finaldat.to_csv(os.path.join(nest_folder, "winks_finaldat.csv"))

In [ ]:
jayew_finaldat = compile_dat('da052c14', 'Jordan-Ayew', 'Jordan_Ayew', checkgames = True)
if isinstance(jayew_finaldat, pd.DataFrame):
    jayew_finaldat.to_csv(os.path.join(nest_folder, "jayew_finaldat.csv"))

In [ ]:
skipp_finaldat = compile_dat('6250083a', 'Oliver-Skipp', 'Oliver_Skipp', checkgames = True)
if isinstance(skipp_finaldat, pd.DataFrame):
    skipp_finaldat.to_csv(os.path.join(nest_folder, "skipp_finaldat.csv"))

In [ ]:
soumare_finaldat = compile_dat('ae59b359', 'Boubakary-Soumare', 'Boubakary_Sou', checkgames = True)
if isinstance(soumare_finaldat, pd.DataFrame):
    soumare_finaldat.to_csv(os.path.join(nest_folder, "soumare_finaldat.csv"))

In [ ]:
choudhury_finaldat = compile_dat('7d2d3329', 'Hamza-Choudhury', 'Hamza_Choudhury', checkgames = True)
if isinstance(choudhury_finaldat, pd.DataFrame):
    choudhury_finaldat.to_csv(os.path.join(nest_folder, "choudhury_finaldat.csv"))

## Liverpool

In [ ]:
szoboszlai_finaldat = compile_dat('934e1968', 'Dominik-Szoboszlai', 'Dominik_Szoboszlai', checkgames = True)
if isinstance(szoboszlai_finaldat, pd.DataFrame):
    szoboszlai_finaldat.to_csv(os.path.join(nest_folder, "szoboszlai_finaldat.csv"))

In [ ]:
macallister_finaldat = compile_dat('83d074ff', 'Alexis-Mac-Allister', 'Alexis_Mac', checkgames = True)
if isinstance(macallister_finaldat, pd.DataFrame):
    macallister_finaldat.to_csv(os.path.join(nest_folder, "macallister_finaldat.csv"))

In [ ]:
gravenberch_finaldat = compile_dat('b8e740fb', 'Ryan-Gravenberch', 'Ryan_Gravenberch', checkgames = True)
if isinstance(gravenberch_finaldat, pd.DataFrame):
    gravenberch_finaldat.to_csv(os.path.join(nest_folder, "gravenberch_finaldat.csv"))

In [ ]:
elliott_finaldat = compile_dat('b9e1436c', 'Harvey-Elliott', 'Harvey_Elliott', checkgames = True)
if isinstance(elliott_finaldat, pd.DataFrame):
    elliott_finaldat.to_csv(os.path.join(nest_folder, "elliott_finaldat.csv"))

In [ ]:
curtis_finaldat = compile_dat('4fb9c88f', 'Curtis-Jones', 'Curtis_Jones', checkgames = True)
if isinstance(curtis_finaldat, pd.DataFrame):
    curtis_finaldat.to_csv(os.path.join(nest_folder, "curtis_finaldat.csv"))

In [ ]:
endo_finaldat = compile_dat('c149016b', 'Wataru-Endo', 'Wataru_Endo', checkgames = True)
if isinstance(endo_finaldat, pd.DataFrame):
    endo_finaldat.to_csv(os.path.join(nest_folder, "endo_finaldat.csv"))

## Manchester City